# 📘 06 - Regex Validators in Pydantic

Pydantic supports powerful regex-based validation for string fields.

---

## 🔍 Why Regex Validation?

Regex (Regular Expressions) allow you to define **text patterns** — great for:
- Email, username, phone number formats
- Password strength
- Custom ID formats

In **Pydantic v2**, use `pattern=` inside `Field(...)` to enforce a pattern.

You can also create **custom validators** using the `@field_validator` decorator if your logic is more complex.

In [1]:
# Import
from pydantic import BaseModel, Field

## ✅ 1. Pattern Matching with `pattern=`

Let’s validate an email field using a regex pattern (Pydantic v2).

In [2]:
# Email Pattern Example
class User(BaseModel):
    username: str
    email: str = Field(pattern=r"^\S+@\S+\.\S+$")  # simple email regex

# Valid case
user = User(username="gigi", email="gigi@example.com")
print(user)

# Invalid email
try:
    bad = User(username="gigi", email="invalid-email")
except Exception as e:
    print(e)

username='gigi' email='gigi@example.com'
1 validation error for User
email
  String should match pattern '^\S+@\S+\.\S+$' [type=string_pattern_mismatch, input_value='invalid-email', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_pattern_mismatch


## ☎️ 2. Custom Regex with `pattern` — Phone Number

We can validate phone numbers using a specific pattern like `+91-XXXXXXXXXX`.

In [3]:
# Phone Pattern
class Contact(BaseModel):
    name: str
    phone: str = Field(pattern=r"^\+91-\d{10}$")  # e.g., +91-9876543210

# Valid number
print(Contact(name="GiGi", phone="+91-9876543210"))

# Invalid number
try:
    wrong = Contact(name="Gireesh", phone="98765")
except Exception as e:
    print(e)

name='GiGi' phone='+91-9876543210'
1 validation error for Contact
phone
  String should match pattern '^\+91-\d{10}$' [type=string_pattern_mismatch, input_value='98765', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_pattern_mismatch


## 🔐 3. Custom Validator for Password Strength

Use `@field_validator` when regex is part of custom logic — like password rules:
- 8+ chars
- At least one uppercase
- One number
- One special char

In [4]:
# Custom Password Validator
import re
from pydantic import field_validator

class SignUpModel(BaseModel):
    username: str
    password: str

    @field_validator("password")
    def check_password_strength(cls, value):
        if len(value) < 8:
            raise ValueError("Password must be at least 8 characters.")
        if not re.search(r"[A-Z]", value):
            raise ValueError("Must include at least one uppercase letter.")
        if not re.search(r"\d", value):
            raise ValueError("Must include at least one digit.")
        if not re.search(r"[!@#$%^&*()_+]", value):
            raise ValueError("Must include a special character.")
        return value

# Test it
try:
    user = SignUpModel(username="gigi", password="Pass123!")
    print(user)
except Exception as e:
    print(e)

username='gigi' password='Pass123!'


## 🧾 Summary

| Method                      | Use Case                                |
|----------------------------|------------------------------------------|
| `pattern=` in `Field()`     | Simple pattern match (e.g. email, phone) |
| `@field_validator`          | Complex patterns, multiple checks        |
| `re` module                 | For full regex control inside validators |

Regex helps prevent invalid or malicious input before your logic even runs. Combine `pattern` and custom validators for best results. 💪